In [3]:
import pandas as pd
import nltk
from sklearn.utils import shuffle

# Loading our datafile as a DataFrame with pandas

df = pd.read_csv('/Users/laniepreston/TMCI_Edits/group_assignment/FullReviews.csv' , usecols=[1,6,9], header=0)
shuf_red_df = shuffle(df[:10000])
print(shuf_red_df)

       ProductId  Score                                               Text
7812  B00474H936      4  I have tried almost every kind of "fake meat" ...
2304  B0002Z9BF8      5  <a href="http://www.amazon.com/gp/product/B000...
5224  B0009OMU00      5  I tasted this snack at my local pub and found ...
7062  B004K30HO2      1  I was so excited to get these k cups and immed...
7105  B004K30HO2      4  I had been looking for something to use with m...
3342  B005K4Q1VI      5  This is the absolute best hot cocoa for the Ke...
8473  B003VXFK44      5  It was a bit of a risk to order this, I mean w...
5999  B003SE52K8      5  We tried all flavors for our cats and the rabb...
7748  B001E5DZTS      5  This is a really good source for goats milk. I...
907   B000ER6YO0      4  I like the Earth's Best baby food line because...
5735  B003TIVBSU      5  I don't understand(believe) the ONE negative r...
7591  B000CQ6KSI      5  No long, frightening lists of chemical additiv...
7500  B000S11CD0      5  

In [15]:
# PoS-tagging our dataset
train_df = shuf_red_df[:8000]
test_df = shuf_red_df[:2000]


train_tagged = []
test_tagged = []


for index, item in train_df.iterrows():
    product_id = item[0]
    score = item[1]
    review = item[2]
    review = review.split()
    rev = []
    for i in review:
        rev += nltk.pos_tag([i], tagset="universal")
    train_tagged.append([rev, score])

for index, item in test_df.iterrows():
    product_id = item[0]
    score = item[1]
    review = item[2]
    review = review.split()
    rev = []
    for i in review:
        rev += nltk.pos_tag([i], tagset="universal")
    test_tagged.append([rev, score])
    

In [11]:
cooccs_text = []
for index, item in shuf_red_df.iterrows():
    product_id = item[0]
    score = item[1]
    review = item[2]
    review = review.split()
    rev = []
    for i in review:
        rev += nltk.pos_tag([i], tagset="universal")
        cooccs_text.append([product_id, rev, score])

In [22]:
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
# Lemmatizing our dataset + filtering for stopwords (except negation)
training_dicts = dict((k,[]) for k in [1,2,3,4,5])
test_dicts = dict((k,[]) for k in [1,2,3,4,5])

stopwords = set(stopwords.words('english'))

un2wn_mapping = {"VERB" : wn.VERB, "NOUN" : wn.NOUN, "ADJ" : wn.ADJ, "ADV" : wn.ADV}

lemmatized = []

for review in train_tagged:
    rev = review[0]
    score = review[1]
    lem_rev = []
    for tup in rev:
        w, tag = tup
        
        if len(w) > 1 and w.isalpha():
            w = w.lower()
        else:
            continue
        
        if tag in [".", "X"]:
            continue
        elif w.lower() in stopwords:
            if w.lower() in ["not", "t", "no"]:  
                lemma = w.lower()
                tag = "NEGATION"
            else: 
                continue
        
        elif tag in un2wn_mapping.keys():
            lemma = nltk.WordNetLemmatizer().lemmatize(w, pos = un2wn_mapping[tag])
        else:
            lemma = nltk.WordNetLemmatizer().lemmatize(w)
        lem_rev.append("-".join([lemma, tag]))
    training_dicts[score].append(lem_rev)

In [24]:
for review in test_tagged:
    rev = review[0]
    score = review[1]
    lem_rev = []
    for tup in rev:
        w, tag = tup
        
        if len(w) > 1 and w.isalpha():
            w = w.lower()
        else:
            continue
        if tag in [".", "X"]:
            continue
        elif w.lower() in stopwords:
            if w.lower() in ["not", "t", "no"]:  
                lemma = w.lower()
                tag = "NEGATION"
                lem_rev.append("-".join([lemma, tag]))
            else: 
                continue
        elif tag in un2wn_mapping.keys():
            lemma = nltk.WordNetLemmatizer().lemmatize(w, pos = un2wn_mapping[tag])
        else:
            lemma = nltk.WordNetLemmatizer().lemmatize(w)
        lem_rev.append("-".join([lemma, tag]))
    test_dicts[score].append(lem_rev)

In [17]:
cooccs_dict = {}
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
stopwords = set(stopwords.words('english'))

un2wn_mapping = {"VERB" : wn.VERB, "NOUN" : wn.NOUN, "ADJ" : wn.ADJ, "ADV" : wn.ADV}

for review in cooccs_text:
    product_id = review[0]
    rev = review[1]
    score = review[2]
    lem_rev = []
    
    for tup in rev:
        w, tag = tup
        
        if len(w) > 1 and w.isalpha():
            w = w.lower()
        else:
            continue
        if tag in [".", "X"]:
            continue
        elif w.lower() in stopwords:
            if w.lower() in ["not", "t", "no"]:  
                lemma = w.lower()
                tag = "NEGATION"
                lem_rev.append("-".join([lemma, tag]))
            else: 
                continue
        elif tag in un2wn_mapping.keys():
            lemma = nltk.WordNetLemmatizer().lemmatize(w, pos = un2wn_mapping[tag])
        else:
            lemma = nltk.WordNetLemmatizer().lemmatize(w)
        lem_rev.append("-".join([lemma, tag]))
        
    cooccs_dict[product_id] = (score, lem_rev)

In [28]:
import pickle
# Save lemmatized text to a pickle file
pickle.dump(training_dicts, open("training_dicts.txt", "wb"))
pickle.dump(test_dicts, open("test_dicts.txt", "wb"))
pickle.dump(cooccs_dict, open("cooccs_dict.txt", "wb"))

In [18]:
print(cooccs_text[:5])

[['B00474H936', [('I', 'PRON'), ('have', 'VERB'), ('tried', 'VERB'), ('almost', 'ADV'), ('every', 'DET'), ('kind', 'NOUN'), ('of', 'ADP'), ('"fake', 'NOUN'), ('meat"', 'NOUN'), ('product', 'NOUN'), ('out', 'ADP'), ('there.', 'NOUN'), ('While', 'ADP'), ('I', 'PRON'), ('really', 'ADV'), ('like', 'ADP'), ('the', 'DET'), ('Morning', 'VERB'), ('Stars', 'NOUN'), ('and', 'CONJ'), ('the', 'DET'), ('Boca', 'NOUN'), ('"ground', 'NOUN'), ('meat"', 'NOUN'), ('you', 'PRON'), ("can't", 'NOUN'), ('really', 'ADV'), ('make', 'VERB'), ('them', 'PRON'), ('into', 'ADP'), ('anything.<br', 'NOUN'), ('/><br', 'NOUN'), ('/>This', 'NOUN'), ('product', 'NOUN'), ('was', 'VERB'), ('great.', 'NOUN'), ('Soak', 'NOUN'), ('it', 'PRON'), ('for', 'ADP'), ('15', 'NUM'), ('minutes', 'NOUN'), ('in', 'ADP'), ('water,', 'NOUN'), ('and', 'CONJ'), ('viola', 'NOUN'), ('you', 'PRON'), ('have', 'VERB'), ('a', 'DET'), ('really', 'ADV'), ('good', 'ADJ'), ('ground', 'NOUN'), ('meat', 'NOUN'), ('you', 'PRON'), ('can', 'VERB'), ('mak